In [102]:
# !pip install panel
# !pip install jupyter_bokeh
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install panel
# !pip install jupyter_bokeh


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [103]:
# import necessary libraries and functions  to load and visualize the data
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import panel as pn
import param
pn.extension()

# from pandas.plotting import scatter_matrix

In [104]:
# Load the dataset
df = pd.read_csv('microsoft_stock.csv')

# Display the first 10 rows of the dataset
df.head(10)

,Date,Open,High,Low,Close,Volume
0,4/1/2015 16:00:00,40.60,40.76,40.31,40.72,36865322
1,4/2/2015 16:00:00,40.66,40.74,40.12,40.29,37487476
2,4/6/2015 16:00:00,40.34,41.78,40.18,41.55,39223692
3,4/7/2015 16:00:00,41.61,41.91,41.31,41.53,28809375
4,4/8/2015 16:00:00,41.48,41.69,41.04,41.42,24753438
5,4/9/2015 16:00:00,41.25,41.62,41.25,41.48,25723861
6,4/10/2015 16:00:00,41.63,41.95,41.41,41.72,28022002
7,4/13/2015 16:00:00,41.40,42.06,41.39,41.76,30276692
8,4/14/2015 16:00:00,41.80,42.03,41.39,41.65,24244382
9,4/15/2015 16:00:00,41.76,42.46,41.68,42.26,27343581


I will utilise the microsoft Stock dataset I have used in my time series project. e.g. easier to follow on from

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1511 non-null   object 
 1   Open    1511 non-null   float64
 2   High    1511 non-null   float64
 3   Low     1511 non-null   float64
 4   Close   1511 non-null   float64
 5   Volume  1511 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 71.0+ KB


In [106]:
#Check again for missing values, (I have no missing values)
print("Number of Null Values:",df.isnull().sum())

Number of Null Values: Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [107]:
# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [108]:
#Check for duplicate values
print("Number of Duplicated Values: ",df.duplicated().sum())

Number of Duplicated Values:  0


## Wireframe and Placeholder design thinking

I decided upon using Panel as I found it easier to follow and also similar to what I normally use(PowerBI).

I created a 8 panel dashboard, with 4 placeholders for the plots, and 1 placeholders for the text/insights. 

I intially thought it would be a 4 panel but with placement of the plots, it was better to have 8 panels.

The layout of the grid honeslty has changed multiple times to ensure I fitted all the panels in a strutured way, intially it was 2x2 panel,

In [109]:
# Create wireframe with placeholders, with background colours for ease of visility.
Plot = pn.pane.Markdown("## Plot", style={'color': 'white'}, background='black')
text = pn.pane.Markdown("## Text/Insights", style={'color': 'white'}, background='black')
table = pn.pane.Markdown("## Table", style={'color': 'white'}, background='black')

# Create a grid layout for the dashboard
dashboard_layout = pn.GridSpec(sizing_mode='stretch_both')

# Populate the grid with placeholders
dashboard_layout[0, 0] = Plot #interactive_dashboard
dashboard_layout[0, 1] = Plot #Box plot small rectangle
dashboard_layout[3, 1] = Plot #histogram 
dashboard_layout[3, 0] = Plot #heatmap
dashboard_layout[2, 1] = text  # Move placeholder_text to the bottom center

# Showing the dashboard in notebook
dashboard_layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'39f0e135-2cb2-49a0-b307-912231e2d768': {'version…

## Create Plots

I Initially create plots for each of the 8 plots as per brief but quickly realsied that I needed to be selective.
I choose 
1. Heatmap of all features
2. Scatter plot all features
3. Interactive Histogram of all features for easier exploration
4. Interactive Box Plot for distribtion
5. Line chart which would be my landing panel (0,0)

I used go.figure and plotly to create all the dashboards again as it was easier to maniuplate sizing etc.

I also wanted to create interactive controls for the user so that they could 'Interactive Analysis of Microsoft Stock Prices' over time.

In [110]:
# Calculating correlation for heatmap
correlation_matrix = df[['Open', 'High', 'Low', 'Close', 'Volume']].corr()

# Round the values in the correlation matrix to 4 decimal places
correlation_matrix = correlation_matrix.round(4)

# create labels for the heatmap
x_labels = y_labels = correlation_matrix.columns.tolist()

# Create the heatmap
fig = ff.create_annotated_heatmap(z=correlation_matrix.values, x=x_labels, y=y_labels, colorscale='RdBu')
fig.update_layout(title_text='Heatmap of Correlation Matrix', title_x=0.4, width=400, height=400)

# Show the plot
fig.show()

# Convert the plotly figure to a Panel pane
heatmap_pane = pn.pane.Plotly(fig)

# Create a grid layout for the dashboard
dashboard_layout = pn.GridSpec(sizing_mode='stretch_both')

In [111]:
# Scatter plot for the distribution of 'Close' price
fig = go.Figure()

for feature in ['Open', 'High', 'Low', 'Close']:
    fig.add_trace(go.Scatter(y=df[feature], mode='markers', name=feature))

fig.update_layout(
    autosize=False,
    width=600,
    height=400,
    title_text="Scatter Plot of Prices"
)
fig.show()

# Convert the Plotly figure to a Panel object
scatter_plot_pane = pn.pane.Plotly(fig)

In [112]:
# Histograms for each continuous variable
fig = go.Figure()
for feature in ['Open', 'High', 'Low', 'Close']:
    fig.add_trace(go.Histogram(x=df[feature], name=feature, nbinsx=20))

fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    barmode='overlay',
    title_text='Histograms',
    xaxis_title_text='Value',
    yaxis_title_text='Count',
    bargap=0.2
)
fig.update_traces(opacity=0.75)
fig.show()

# Convert the Plotly figure to a Panel object
histogram_pane = pn.pane.Plotly(fig)

In [113]:
# Box plot for the distribution of 'Close' price
fig = go.Figure()

for feature in ['Open', 'High', 'Low', 'Close']:
    fig.add_trace(go.Box(y=df[feature], name=feature))

fig.update_layout(
    autosize=False,
    width=600,
    height=400,
    title_text="Box Plot of Prices"
)
fig.show()

# Convert the Plotly figure to a Panel object
box_plot_pane = pn.pane.Plotly(fig)

In [114]:
# Line chart for 'Close' price over time
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Close Price'))

fig.update_layout(
    autosize=False,
    width=700,
    height=500,
    title='Closing Price of Microsoft Stock Over Time',
    xaxis_title='Date',
    yaxis_title='Close Price'
)

fig.show()

In [115]:
# https://builtin.com/data-science/create-dashboards-panel-python
# https://oceanhackweek.org/ohw22/tutorials/02-Wed/01-data-visualization-in-python/tutorial/02_Building_Panels.html

# Creating the interactive controls
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider', 
    start=df['Date'].min(), 
    end=df['Date'].max(),
    value=(df['Date'].min(), df['Date'].max())
)

# Create a string pane to display the selected date range
date_range_pane = pn.pane.Str()

# Create a pane to display the plot
plot_pane = pn.pane.Plotly()

# Function to create a plot
def create_plot(data):
    fig = go.Figure(data=go.Scatter(x=data['Date'], y=data['Close'], mode='markers'))
    return fig

# Function to dynamically update the date range string and plot based on the slider value
# https://panel.holoviz.org/user_guide/Param.html
import datetime

@param.depends(date_range_slider.param.value)
def update_date_range(value):
    start_date, end_date = value
    start_date = pd.Timestamp(datetime.datetime.combine(start_date, datetime.datetime.min.time()))
    end_date = pd.Timestamp(datetime.datetime.combine(end_date, datetime.datetime.max.time()))
    
    date_range_pane.object = f"Selected date range: {start_date} to {end_date}"
    
    # Filter the data based on the selected date range
    filtered_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Update the plot
    plot_pane.object = create_plot(filtered_data)

# Organizing the dashboard layout
dashboard_layout = pn.Column(
    pn.pane.Markdown("# Microsoft Stock Dashboard"),
    pn.pane.Markdown("## Interactive Analysis of Microsoft Stock Prices"),
    date_range_slider,
    date_range_pane,
    plot_pane,
    update_date_range # This is the function that will be called when the slider value changes
)

### Final Grid Panel Layout
1. interactive 
2. Plot Box plot
3. Histogram
4. Heatmap
5. Adding text to panel

In [116]:
# Create a Markdown pane
text = pn.pane.Markdown("")

# Set the height of the text pane
text.height = 50

# Enter text in the text box with Markdown formatting
text.object = """
1. Please scroll up and down to view all Panels
2. Use the slider to select a date range
3. The Target Variables are 'Open', 'High', 'Low', 'Close', 'Volume'
4. The Dataset is highly Correlated
5. The Histograms show the distribution of the target variables
6. The Box Plot shows the distribution of the target variables
7. The Line Chart shows the closing price of Microsoft stock over time
"""

In [117]:
# Organizing the dashboard layout
interactive_dashboard = pn.Column(
    pn.pane.Markdown("# Microsoft Stock Dashboard"),
    pn.pane.Markdown("## Closing Price of Microsoft Stock Over Time"),
    date_range_slider,
    date_range_pane,
    plot_pane,
    update_date_range  # This is the function that will be called when the slider value changes
)

# Create a grid layout for the dashboard
grid_layout = pn.GridSpec(sizing_mode='stretch_both')

# Populate the grid with placeholders
grid_layout[0, 0] = interactive_dashboard
grid_layout[0, 1] = box_plot_pane
grid_layout[3, 1] = histogram_pane
# grid_layout[3, 0] = heatmap_pane
grid_layout[3, 0] = scatter_plot_pane
grid_layout[2, 1] = text 

# Display the dashboard as a Dasboard Web App
pn.serve(grid_layout, show=True)

Launching server at http://localhost:49300
